In [16]:
%%writefile C:\Anaconda\Lib\NewCAMP.py
    
import datetime as dt
import pandas as pd
import numpy as np
import math as math 
import MathsUtilities as MUte
from scipy.optimize import curve_fit

CampConstants = {
    'k':-0.17,
    'VSThreshold':1.0,
    'TSThreshold':2.0,
    'SlopeFLNvsTS':1.1,
    'IntFLNvsTS':2.85,
    'PpMax':20,
    'PpMin':8,
}

def CalcdHS(Tt,Pp,HS,BasePhyllo,PhylloPpSens):
    """Calculate daily increase in Haun stage
    Args:
        Tt: Thermal time increment
        Pp: The current photoperiod
        HS: Current Haun Stage
        BasePhyllo: Base phyllochron at 16h Pp between 3 and 7 HS
        PhylloPpSens: Photo period sensitivity of phyllochron.  Relative increase in phyllochron as 8h Pp
    Returns:
        Daily increas in Haun Stage
    """
    HS = max(0,HS)
    StageFactor = np.interp(HS,[0,2,2.0001,7,7.0001,12],[0.75,0.75,1.0,1.0,1.4,1.4],False)
    PpFactor = 1 + PhylloPpSens * np.interp(Pp,[0,8,12,20],[1,1,0,0],False)
    Phyllochron = BasePhyllo*StageFactor*PpFactor
    return Tt/Phyllochron

def CalcPpResponse(Pp, Base, Max):
    """ Calculate value of a photoperiod sensitive variable between Max and Baee values
    Args:
        Pp: Photoperiod
        Base: value below 8h Pp
        Max: value above 16h Pp
        dPB: delta base phyllochrons
    Returns:
        variable value dependent on photoperiod and adjusted for dHS
    """
    if Pp <= CampConstants['PpMin']:
        return Base
    if (Pp > CampConstants['PpMin']) and (Pp < CampConstants['PpMax']):
        return (Base + (Max-Base) * (Pp-CampConstants['PpMin'])/(CampConstants['PpMax']-CampConstants['PpMin']))
    if (Pp >= CampConstants['PpMax']):
        return Max

def CalcBaseUpRegVrn1(Tt, dHS, BaseDVrn1):
    """ Calculate upregulation of base Vrn1
    
    Args:
        Tt: Thermal time increment
        dHS: Delta HaunStage
        BaseDVrn1: coeffociennt for Base Vrn1 expression
        
    Returns:
        delta BaseVrn1 representing the additional Vrn1 expression from base expression
    """
    if Tt < 0: 
        BaseDVrn1 = 0
    return BaseDVrn1 * dHS

def CalcColdUpRegVrn1(Tt,dHS, MaxDVrn1, k):
    """ Upregulation of Vrn1 from cold.  Is additional to base vrn1
        BaseDVrn1 in seperate calculation otherwise te same as Brown etal 2013
    Args:
        Tt: Thermal time increment
        dHS: Delta Base Phyllochrons
        MaxDVrn1: coefficient for Maximum upregulation of Vrn1
        k: The exponential coefficient determining temperature response
        
    Returns:
        delta ColdVrn1 representing the additional Vrn1 expression from cold upregulation
    """
    UdVrn1 = MaxDVrn1 * np.exp(k*Tt)
    if (Tt < 20):
        return UdVrn1 * dHS
    else:
        return -5
        
def calcTSHS(FLN,IntFLNvsTSHS,SlopeFLNvsTS):
    """Haun stage timing of terminal spikelet 
       Inverts equation 5 from Brown etal 2013 FLN =  2.85 + 1.1*TSHS and converts it to base phyllochrons.
       Note the intercept differs, was typeo on publication
       Intercept has been made variable for as needs to be lower for some very fast varieties
    Args:
        FLN: The final leaf number
        IntFLNvsTSHS: The intercept of the regression between FLN and TSHS
    Returns:
        Estimation of number of base phyllochrons to terminal spikelet
    """
    return (FLN - IntFLNvsTSHS)/SlopeFLNvsTS

def CAMPmodel(Out, Day, Tt, Pp, Params, Consts, TtEmerge):
    """ The Cereal Anthesis Molecular Phenology model.
        Based on the ideas presented in Brown etal 2014 (Annals of Botany)
        Alterations made replacing Vrn4 notion with methalation of Vrn1, introducing VrnX to account for long day vernalisation
        Vrn2 approach changed to have hihger experssion for first HS then a lower rate
        Other alterations to implement working code base
    Args:
        Out: "FLN" or None.  If "FLN" will only return estimated FLN else will return full dataframe with daily state variable values
        Day: List, 1:EndDay representing the timesteps in model run
        Tt: List of same length as Day, representing daily temperature
        Pp: List of same length as Day, representing daily photoperiod
        Params: Dict with genotype parameters fitted by CalcCultivarVrnCoeffs() 
        Consts: Dict with crop specific constants,
        TtEmerge: Thermaltime from sowing to emergence
    """
    # Set up Data structure and initialise values
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    DF = pd.DataFrame(index = [0], columns = ['Day','Tt','Pp'])
    IsGerminated = False
    IsEmerged = False
    IsMethalating = False
    IsVernalised = False
    IsReproductive = False
    IsAtFlagLeaf = False
    DF.loc[0,'Day'] = 0
    DF.loc[0,'Tt'] = Tt[0]
    DF.loc[0,'RelCold'] = 0
    DF.loc[0,'Pp'] = Pp[0]
    DF.loc[0,'RelPp'] = 0
    DF.loc[0,'Stage'] = 'Germination'
    DF.loc[0,'IsGerminated'] =True
    DF.loc[0,'IsEmerged'] = False
    DF.loc[0,'IsVernalised'] = False
    DF.loc[0,'IsReproductive'] = False
    DF.loc[0,'IsAtFlagLeaf'] = False
    DF.loc[0,'HS'] = -TtEmerge/Params.BasePhyllo
    DF.loc[0,'dHS'] = 0
    DF.loc[0,'AccumTt'] = Tt[0]
    DF.loc[0,'BaseVrn'] = 0
    DF.loc[0,'dBaseVrn'] = 0
    DF.loc[0,'MaxVrn'] = 0
    DF.loc[0,'dMaxVrn'] = 0
    DF.loc[0,'Cold'] = 0
    DF.loc[0,'dCold'] = 0
    DF.loc[0,'Vrn'] = 0
    DF.loc[0,'MaxVrn2'] = Params.MaxVrn2
    DF.loc[0,'Vrn2'] = 0.0
    DF.loc[0,'Vrn1'] = 0.0
    DF.loc[0,'dVrn1'] = 0
    DF.loc[0,'Vrn3'] = 0.0
    DF.loc[0,'dVrn3'] = 0
    DF.loc[0,'VSHS'] = 0
    DF.loc[0,'TSHS'] = 0
    DF.loc[0,'FLN'] = Consts['IntFLNvsTS']
    
    d = 1
    
    # Model daily loop
    # ^^^^^^^^^^^^^^^^
    while (IsAtFlagLeaf == False) and (d < Day[-1]):
        ColdYesterday = 0
        ColdYesterday = DF.loc[d-1,'Cold']
        DF.loc[d,:] = DF.loc[d-1,:] #Carry yesterdays values over to today
        DF.loc[d,'Stage'] = np.nan
            
        DF.loc[d,'Day'] = d
        # Set daily environment variables
        DF.loc[d,'Tt'] = Tt[d]
        DF.loc[d,'Pp'] = Pp[d]
        #Zero set deltas
        DF.loc[d,'dHS'] = 0
        DF.loc[d,'dBaseVrn'] = 0
        DF.loc[d,'dMaxVrn'] = 0
        DF.loc[d,'dCold'] = 0
        DF.loc[d,'Vrn2'] = 0
        DF.loc[d,'dVrn1'] = 0
        DF.loc[d,'dVrn3'] = 0
        
                
        DF.loc[d,'AccumTt'] = DF.loc[d,'AccumTt'] + DF.loc[d,'Tt']
        
        PropnOfDay = 1.0
        if (DF.loc[d,'AccumTt'] > TtEmerge) and (IsEmerged==False):
            IsEmerged = True
            DF.loc[d,'Stage'] = 'Emergence'
            PropnOfDay = (DF.loc[d,'AccumTt'] - TtEmerge)/DF.loc[d,'Tt'] # Calculate fraction of emergence days Tt that is not used for emergence
        
        # Calculate daily Haun Stage changes
        # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        if IsEmerged==False: # Crop not yet emerged 
            EmergDurationFactor = 1
            if (DF.loc[d,'AccumTt'] > 90):  # Calculate EmergenceDurationFactor to slow accumulation of HS if emergence is taking a long time.  This slows Vrn1 expression under slow emergence and strange responses to delayed sowing
                EmergDurationFactor = np.exp(-0.015 * (DF.loc[d,'AccumTt']-90))
            DF.loc[d,'dHS'] = DF.loc[d,'Tt']/Params.BasePhyllo  * EmergDurationFactor
            DF.loc[d,'RelPp'] = 0
            DF.loc[d,'Vrn2'] = 0
            
        else: # Crop emerged
            # Calculate delta haun stage
            DF.loc[d,'dHS'] = CalcdHS(DF.loc[d,'Tt'],DF.loc[d,'Pp'],DF.loc[d,'HS'],Params.BasePhyllo,Params.PhylloPpSens)
            
        #increment HS
        DF.loc[d,'HS'] += DF.loc[d,'dHS']
        
        # Calculate Vrn gene expression
        # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        if (IsVernalised==False): #set as rates and factors for vegetative phase
            DF.loc[d,'dBaseVrn'] = Params.BaseDVrnVeg * DF.loc[d,'dHS']
            DF.loc[d,'dMaxVrn'] = Params.MaxDVrnVeg * DF.loc[d,'dHS']
            DF.loc[d,'PpVrn3Fact'] = Params.PpVrn3FactVeg
        else:
            if (IsReproductive==False): #set as rates and factors for early reproductive phase
                DF.loc[d,'dBaseVrn'] = Params.BaseDVrnER * DF.loc[d,'dHS']
                DF.loc[d,'dMaxVrn'] = Params.MaxDVrnER * DF.loc[d,'dHS']
                DF.loc[d,'PpVrn3Fact'] = Params.PpVrn3FactER
        #Increment BaseVern and MaxVern
        DF.loc[d,'BaseVrn'] += DF.loc[d,'dBaseVrn']
        DF.loc[d,'MaxVrn'] += DF.loc[d,'dMaxVrn']
                                      
        # Calculate daily cold response
        if (IsVernalised==False):
            DF.loc[d,'RelCold'] = CalcColdUpRegVrn1(DF.loc[d,'Tt'],1,1,Consts['k'])
            DF.loc[d,'dCold'] = DF.loc[d,'RelCold'] * DF.loc[d,'dBaseVrn'] * Params.ColdVrn1Fact
            DF.loc[d,'Cold'] = max(0.0,DF.loc[d,'Cold'] + DF.loc[d,'dCold'])
            if DF.loc[d,'Cold'] > Params.MethalationThreshold:
                DF.loc[d,'dVrn1'] = max(0,(DF.loc[d,'Cold'] - Params.MethalationThreshold)-(ColdYesterday - Params.MethalationThreshold))
        # Increment Vrn 1
        DF.loc[d,'Vrn1'] += DF.loc[d,'dVrn1']
        
        # Calcualte expression of photoperiod sensitive genes
        if (IsEmerged==True):  # Photoperiod sensitive genes only express after emergence
            DF.loc[d,'RelPp'] = CalcPpResponse(DF.loc[d,'Pp'],0,1) #relative Pp, scaled between 0 at lower threshold and 1 at upper threshold
            if DF.loc[d,'Pp'] <= Consts['PpMin']: #Reduce MaxVrn2 if short Pp encountered
                DF.loc[d,'MaxVrn2'] -= DF.loc[d,'dBaseVrn']
            if(IsVernalised == False): # Set Vrn2 relative to MaxVrn2 and current Pp
                DF.loc[d,'Vrn2'] = DF.loc[d,'MaxVrn2'] * DF.loc[d,'RelPp']
            if (DF.loc[d,'BaseVrn'] + DF.loc[d,'Vrn1']) > DF.loc[d,'Vrn2']: # express vrn3 relative to Pp if Vrn2 is down regulated
                DF.loc[d,'dVrn3'] = (DF.loc[d,'PpVrn3Fact']-1) * DF.loc[d,'RelPp'] * DF.loc[d,'dBaseVrn']
        #Increment Vrn3
        DF.loc[d,'Vrn3'] += DF.loc[d,'dVrn3']

        # Increment todays Vrn expression values using deltas just calculated
        if (IsVernalised==False): #If not vernalised need to include Vrn2 and constrain to maxrate
            DF.loc[d,'Vrn'] = min(DF.loc[d,'MaxVrn'],max(0,DF.loc[d,'BaseVrn'] + DF.loc[d,'Vrn1'] + DF.loc[d,'Vrn3'] - DF.loc[d,'Vrn2'])) 
        else:
            DF.loc[d,'Vrn'] = DF.loc[d,'Vrn'] + DF.loc[d,'dBaseVrn'] + DF.loc[d,'dVrn3']

        # Set Haun stage variables
        # ^^^^^^^^^^^^^^^^^^^^^^^^
        if IsVernalised == False:
            DF.loc[d,'VSHS'] = DF.loc[d,'HS']
        if IsReproductive == False:
            DF.loc[d,'TSHS'] = DF.loc[d,'HS']
            DF.loc[d,'FLN'] = Consts['IntFLNvsTS'] + Consts['SlopeFLNvsTS'] * DF.loc[d,'TSHS']

        # Finally determine phenological stage
        # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        # Vernalisation saturation occurs when Vrn1 > the vernalisation threshold and Vrn2 expression is zero
        if (IsEmerged==True) and (DF.loc[d,'Vrn'] >= Consts['VSThreshold']) and (IsVernalised==False):
            IsVernalised  = True
            DF.loc[d,'Stage'] = 'Vern Sat'
            #DF.loc[d,'Vrn1'] = 0
        
        if (DF.loc[d,'Vrn'] >=  Consts['TSThreshold']) and (IsReproductive == False):
            IsReproductive = True;
            DF.loc[d,'Stage'] = 'Term Spike'
        
        #Work out if Flag leaf has appeared.
        if DF.loc[d,'HS'] >= DF.loc[d,'FLN']:
            IsAtFlagLeaf = True    
            DF.loc[d,'Stage'] = 'FlagLeaf'
    
        #Add states to dataframe
        IsDayOfEmergence = False
        
        DF.loc[d,'IsGerminated'] = IsGerminated
        DF.loc[d,'IsEmerged'] = IsEmerged
        DF.loc[d,'IsVernalised'] = IsVernalised
        DF.loc[d,'IsReproductive'] = IsReproductive
        DF.loc[d,'IsAtFlagLeaf'] = IsAtFlagLeaf
    
        # Increment day
        d += 1
        
    if (Out == 'FLN') and (IsAtFlagLeaf == True):
        return DF.iloc[-1,:]['FLN']
    else:
        return DF
    
def deriveVrnParams(c):
    data = pd.Series(dtype=np.float64)
    data['RelPp'] = (c['PpL']-CampConstants['PpMin'])/(CampConstants['PpMax']-CampConstants['PpMin'])
    # Calculate the accumulated base phyllochrons at terminal spikelet for each treatment
    for pv in ['SN','LV','SV','LN']:
        data['TS_'+pv] = calcTSHS(c['FLN_'+pv],CampConstants['IntFLNvsTS'],CampConstants['SlopeFLNvsTS'])
    # Base Phyllochron
    data['BasePhyllo'] = c['BasePhyllo']
    # Phyllochron photoperiod sensitivity.  #This is not used in calculations but added so can be checked in outputs
    data['PhylloPpSens'] = c['PhylloPpSens']
    # Base Phyllochron duration of the Emergence Phase
    data['EmergDurat'] = c['TtEmerge']/data['BasePhyllo']
    # Base Phyllochron duration of vernalisation treatment
    data['VernTreatDurat'] = (c['VrnTreatDuration']*c['VrnTreatTemp'])/data['BasePhyllo']
    # Accumulated Base Phyllochrons when vernalisation treatment ended 
    data['EndVernTreat'] =  -data['EmergDurat'] + data['VernTreatDurat']
    # The minimum accumulated haunstage at which Vern saturation can occur
    data['MinVS'] = 1.1
    # The minimum haun stage duration from vern saturation to terminal spikelet
    data['MinVS->TS'] = min(3,data['TS_LV']-data['MinVS'])  #From Lincoln controlled environment data
    
    # Calculate the accumulated base phyllochrons at vernalisation saturation for each treatment
    data['VS_LV'] = data['TS_LV'] - data['MinVS->TS']   
    data['VS_LN'] = max(data['TS_LN'] - data['MinVS->TS'],data['VS_LV']) #Constrained so not earlier than the LV treatment
    data['VS_SV'] = min(data['VS_LV'],data['TS_SV']-data['MinVS->TS']) # Assume happens at the same time as VS_LV but not sooner than TS and minVS->TS would allow 
    data['VS->TS_SV'] = data['TS_SV'] - data['VS_SV']
    data['VS_SN'] = max(data['TS_SN'] - data['VS->TS_SV'],data['VS_LV']) #Constrained so not earlier than the LV treatment
    
    # Calculate base and maximum rates and Pp sensitivities
    # Base Vrn delta during vegetative phase.  Assuming base Vrn expression starts at sowing and reaches 1 at VS for the SN treatment where no cold or Vrn3 (short Pp) to upregulate 
    data['BaseDVrnVeg'] = 1/(data['VS_SN']+data['EmergDurat'])
    # The fastest rate that Vrn can accumulate to reach saturation (VS).
    data['MaxDVrnVeg'] = 1/(data['VS_SV']+data['EmergDurat'])
    # Base Vrn delta during early reproductive phase.  Assuming Vrn expression increases by 1 between VS and TS and proceeds at a base rate where not Vrn3 upregulation (short PP)
    data['BaseDVrnER'] = 1/(data['TS_SN']-data['VS_SN'])
    # The maximum Vrn delta during the early reproductive phase.  Assuming Vrn increases by 1 bewtween VS and TS and proceeds at the maximum rate under long photoperiod
    data['MaxDVrnER'] = 1/data['MinVS->TS']
    # The relative increase in delta Vrn cuased by Vrn3 expression under long photoperiod during early reproductive phase
    data['PpVrn3FactER'] = (data['MaxDVrnER']/data['BaseDVrnER'] -1) * 1/data['RelPp'] + 1
    # The relative increase in delta Vrn1 caused by Vrn3 expression under long Pp during vegetative phase.  Same as PpVrn3FactER unless VS_LN is small
    data['PpVrn3FactVeg'] = ((1/data['VS_LN'])/data['BaseDVrnVeg'] -1) * 1/data['RelPp'] + 1
    data['PpVrn3FactVeg'] = max(data['PpVrn3FactER'],data['PpVrn3FactVeg']) 
    
    # Calculate Maximum Vrn2 expression under long photoperiods
    # The rate of Vrn expression before VS under long Pp when Vrn2 = 0 is the product of Vrn3 and base Vrn
    data['dvrnxVrn3'] = max(1/data['VS_LN'],data['BaseDVrnVeg'] * data['PpVrn3FactVeg'] )
    # Therefore the haun stage duration when effective baseVrn x Vrn3 is up regulating during the vegetative phase under long photoperiod without vernalisation is a funciton of the VS target (1) and the rate
    data['vrnxVrn3Durat'] = 1/data['dvrnxVrn3']
    # The accumulated Haun stages when Vrn2 expression is ended and effective baseVrn x Vrn3 expression starts under long Pp without vernalisation is vrn1xVrn3Durat prior to VS
    data['endVrn2_LN'] = max(0,data['VS_LN'] - data['vrnxVrn3Durat']) 
    # The Vrn2 Expression that must be matched by Vrn1 before effective Vrn3 expression starts.  
    data['MaxVrn2'] = (data['endVrn2_LN']+data['EmergDurat']) * data['BaseDVrnVeg']
    
    # Calculate cold upregulation of Vrn1
    # The amount of Vrn expression at base rate in the LV treatment between sowing and when Vrn2 is suppressed 
    data['baseVrnVeg_LV'] = (data['EmergDurat']+data['VS_LV']) * data['BaseDVrnVeg']
    # The amount of persistant (methalated) Vrn1 upregulated due to cold at the end of the vernalisation treatment for LV
    # Is the VS threshold (1) plus the maximum Vrn2 less base vrn expression up to VS and Vrn3 upregulated expression between end of Vrn2 expression and VS
    data['coldVrn1_LV'] = CampConstants['VSThreshold'] + data['MaxVrn2'] - data['baseVrnVeg_LV'] 
    # Cold threshold required before Vrn1 starts methalating.  Based on Brooking and Jamieson data the lag duration is the same length as the duration of response up to full vernalisation
    # Therefore we assume the methalation threshold must be the same size as the amount of vrn1 that is required to give full persistend vernalisation response.
    data['MethalationThreshold'] = data['coldVrn1_LV']
    # Haun stage duration of vernalisation, may be less than treatment duration if treatment goes past VS
    data['vernDurat_LV'] =  min(data['VernTreatDurat'],data['VS_LV']+data['EmergDurat'])
    # The rate of Vrn1 expression under cold treatment calculated from the amopunt of cold vrn1 up regulation apparante plus the methalation threshold
    data['coldDVrn1_LV'] = (data['coldVrn1_LV']+data['MethalationThreshold'])/data['vernDurat_LV'] 
    # The upregulation of Vrn1 expression above base rate at 0oC
    data['coldDVrn1Max'] = data['coldDVrn1_LV']/ np.exp(c['k'] * c['VrnTreatTemp']) 
    # The relative increase in delta Vrn1 caused by cold upregulation of Vrn1
    data['ColdVrn1Fact'] = data['coldDVrn1Max'] /data['BaseDVrnVeg']
                                 
    return data

Overwriting C:\Anaconda\Lib\NewCAMP.py
